In [1]:
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#import pyLDAvis.gensim
import gensim
import pickle 
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.gensim
import os
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

In [2]:
df = pd.read_csv('../data/weeklyRobloxComments.csv')

In [3]:
df.head()

,url,post date,post id,author,date,timestamp,score,comment,comment_id,parent_id
0,https://www.reddit.com/r/roblox/comments/11dr3...,2023-02-28,11dr3u4,lovinmagi,2023-03-02,1.677725e+09,1,there was this game i used to play in 2020 to ...,jaknc17,NaN
1,https://www.reddit.com/r/roblox/comments/11dr3...,2023-02-28,11dr3u4,Nolski77,2023-03-05,1.678051e+09,1,Hello. I’m asking this as a parent trying to k...,jb23etr,NaN
2,https://www.reddit.com/r/roblox/comments/117os...,2023-02-21,117osqh,DavidFromMicrosoft,2023-02-23,1.677182e+09,1,"So, does Roblox on mobile revert it to Dynamic...",j9q5lu1,NaN
3,https://www.reddit.com/r/roblox/comments/117os...,2023-02-21,117osqh,AffectionateYard8591,2023-02-23,1.677194e+09,1,So I was playing on a server with another per...,j9r10yj,NaN
4,https://www.reddit.com/r/roblox/comments/111o7...,2023-02-14,111o7d1,UnrealNL,2023-02-18,1.676708e+09,0,"Hi all,\n\nI am here hoping that I can offer h...",j90jiph,NaN


In [4]:
df['comment']

0      there was this game i used to play in 2020 to ...
1      Hello. I’m asking this as a parent trying to k...
2      So, does Roblox on mobile revert it to Dynamic...
3       So I was playing on a server with another per...
4      Hi all,\n\nI am here hoping that I can offer h...
                             ...                        
507    Apparently accounts are being randomly banned,...
508    I restricted my son's account using the Roblox...
509     Ok so I've come back to roblox after a loooon...
510    did you set a usable email before the PIN was ...
511    > error 267\n\nApparently it happens if your a...
Name: comment, Length: 512, dtype: object

In [5]:
df['comment'].str.count('game').sum()

394

In [6]:
text = " ".join(df['comment'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [7]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
#print(bi_grams_list)

In [8]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: ca nt with a frequency of 59


In [9]:
vectorizer = CountVectorizer(ngram_range = (2, 2))
X1=vectorizer.fit_transform(bi_grams_list1)
features = (vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)


sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                     term  rank
2586               ca nt  61.0
10600  parental controls  41.0
12923     roblox support  38.0
12723     roblox account  29.0
17546           year old  28.0
10123        old account  21.0
8932           make sure  19.0


In [10]:
df_ques = df['comment'].str.endswith('?')
df_true = df.loc[df_ques]
print(df_true['comment'])

3       So I was playing on a server with another per...
6      Parent of a new Roblox player kid, is there an...
11     My 7yr old plays roblox on his fire tablet. Is...
18     I verified my age so I could talk on vc and it...
21     I have a good question for you, How will your ...
                             ...                        
478                                  Are you the parent?
479    Was redirected to post here:\n\nAny way for my...
494    I made some clothing but when I view the cloth...
504    So, my kid started playing Roblox and apparent...
508    I restricted my son's account using the Roblox...
Name: comment, Length: 114, dtype: object


In [11]:
df['text_processed'] = df['comment'].apply(lambda x: re.sub('[,.!?]', '', x) if isinstance(x, str) else x)
df['text_processed'] = df['text_processed'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df['text_processed'].head()


0    there was this game i used to play in 2020 to ...
1    hello i’m asking this as a parent trying to ke...
2    so does roblox on mobile revert it to dynamic ...
3     so i was playing on a server with another per...
4    hi all\n\ni am here hoping that i can offer he...
Name: text_processed, dtype: object

In [12]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.comment.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['game', 'used', 'play', 'early', 'im', 'sure', 'called', 'murder', 'classic', 'version', 'mm', 'voice', 'lines', 'anonymous', 'characters', 'apparently', 'old', 'game', 'seem', 'find', 'anyone', 'know', 'got', 'removed', 'taken', 'replacement']


In [13]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
num_topics = 10
filename = 'WeeklyRobloxComments'
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

LDAvis_data_filepath = os.path.join('../results/'+filename)
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
#s.makedirs('./results/ldavis_prepared_10')
pyLDAvis.save_html(LDAvis_prepared, '../results/'+ filename +'.html')
LDAvis_prepared
pyLDAvis.display(LDAvis_prepared)

c:\Users\mojea\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
